In [6]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news LIMIT 1000")
display(df)

StatementMeta(, 108017ea-3327-454d-8e58-3827c473312f, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a74e6014-5215-459b-a698-df90db73af2c)

In [14]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 108017ea-3327-454d-8e58-3827c473312f, 16, Finished, Available, Finished)

In [15]:
model = (AnalyzeText()
        .setTextCol('description')
        .setKind('SentimentAnalysis')
        .setOutputCol('response')
        .setErrorCol('error'))

StatementMeta(, 108017ea-3327-454d-8e58-3827c473312f, 17, Finished, Available, Finished)

In [18]:
#Apply the Model to df
result = model.transform(df)

StatementMeta(, 108017ea-3327-454d-8e58-3827c473312f, 20, Finished, Available, Finished)

In [20]:
display(result)

StatementMeta(, 108017ea-3327-454d-8e58-3827c473312f, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0bf8a0e8-ab78-4d6d-817f-3fac25985787)

In [21]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df = result.withColumn('Sentiment', col('response.documents.sentiment'))

StatementMeta(, 108017ea-3327-454d-8e58-3827c473312f, 23, Finished, Available, Finished)

In [22]:
display(sentiment_df)

StatementMeta(, 108017ea-3327-454d-8e58-3827c473312f, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cf4e5729-d8fa-4cc0-becb-caf25717f0f6)

In [23]:
sentiment_df_final = sentiment_df.drop('error','response')

StatementMeta(, 108017ea-3327-454d-8e58-3827c473312f, 25, Finished, Available, Finished)

In [24]:
display(sentiment_df_final)

StatementMeta(, 108017ea-3327-454d-8e58-3827c473312f, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0f96488c-e143-4cae-8f61-52112eb4cf67)

In [25]:
from pyspark.sql.utils import AnalysisException

try: 
    
    table_name = 'bing_lake_db.tbl_sentiment_analysis'

    sentiment_df_final.write.format('delta').saveAsTable(table_name)

except AnalysisException:

    print('Table Already Exists')

    sentiment_df_final.createOrReplaceTempView('vw_sentiment_df_final')


    spark.sql(f""" MERGE INTO {table_name} target_table
                   USING vw_sentiment_df_final source_view
                  
                   ON source_view.url = target_table.url

                   WHEN MATCHED and
                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description OR
                   source_view.category <> target_table.category OR
                   source_view.image <> target_table.image OR
                   source_view.provider <> target_table.provider OR
                   source_view.datePublished <> target_table.datePublished

                   THEN UPDATE SET *

                   WHEN NOT MATCHED THEN INSERT *
                   
                """)


StatementMeta(, 108017ea-3327-454d-8e58-3827c473312f, 27, Finished, Available, Finished)